## Obtaining Task Details

The previous chapter#TODOPut link discussed the basic features of RP, how to submit a pilot, and how to submit tasks to that pilot for execution. Here, we show how an application can inspect the details of that execution, after the tasks complete.

You can refer the script 01_task_details.py #TODO put link, which has the following diff to the basic example:

Note that we capture the return value of `submit_tasks()` in line 99#TODO remove line number, which is in fact a list of Task instances. We use those instances for inspection later on, after we waited for their completion. Inspection is also available earlier, but may then yield incomplete results. Note that a task always has a state throughout its life span, according to the state model discussed in RADICAL-Pilot (RP) - Overview.

The code block below shows how to report information about task state, exit code, and standard output. Later, we will see that standard error is handled equivalently.
```shell
report.plain('  * %s: %s, exit: %3s, out: %s\n' \
        % (task.uid, task.state[:4],
            task.exit_code, task.stdout.strip()[:35]))
```

> Reporting standard output in this way is a convenience method that cannot replace proper staging of output files. The string returned by task.stdout.strip()[:35] will be shortened on very long outputs (longer than 1kB by default) and it may contain information from RP which is not part of the standard output of the application. The proper staging of output files will be discussed in a later example.

### Running the Example


In [ ]:
%load_ext dotenv
%dotenv


We start by importing the radical.pilot module and initializing the reporter facility used for printing well formatted runtime and progress information.

In [ ]:
import os
import sys

verbose  = os.environ.get('RADICAL_PILOT_VERBOSE', 'REPORT')
os.environ['RADICAL_PILOT_VERBOSE'] = verbose

import radical.pilot as rp
import radical.utils as ru
# we use a reporter class for nicer output
report = ru.Reporter(name='radical.pilot')
report.title('Getting Started (RP version %s)' % rp.version)



We will set the resource value to 'local.localhost'. Using a resource key other than local.localhost implicitly tells RADICAL-Pilot that it is targeting a remote resource.

In [ ]:
resource = 'local.localhost'


To create a new Session, you need to provide the URL of a MongoDB server which we will fetch from our .env file.
Create a new session. No need to try/except this: if session creation
fails, there is not much we can do anyways...

In [ ]:
session = rp.Session()

All other pilot code is now tried/excepted. If an exception is caught, we can rely on the session object to exist and be valid, and we can thus tear the whole RP stack down via a <i>'session.close()'</i> call in the <i>'finally'</i> clause.

In [ ]:
def initialize_desc_object(resources):
    # read the config used for resource details
    report.info('read config')
    config = ru.read_json('%s/config.json' % os.path.dirname(os.path.abspath(__file__)))
    report.ok('>>ok\n')

    report.header('submit pilots')

    # Define an [n]-core local pilot that runs for [x] minutes
    # Here we use a dict to initialize the description object
    pd_init = {'resource'      : resource,
               'runtime'       : 15,  # pilot runtime (min)
               'exit_on_error' : True,
               'project'       : config[resource].get('project', None),
               'queue'         : config[resource].get('queue', None),
               'access_schema' : config[resource].get('schema', None),
               'cores'         : config[resource].get('cores', 1),
               'gpus'          : config[resource].get('gpus', 0),
               }
    pdesc = rp.PilotDescription(pd_init)


 Add a PilotManager. PilotManagers manage one or more pilots.

In [ ]:
def launch_pilots(session,pdesc):
    pmgr = rp.PilotManager(session=session)
    pilots = pmgr.submit_pilots(pdesc)
    return pilots    

In [ ]:
def submit_tasks(pilots):
    report.header('submit tasks')

    # Register the pilot in a TaskManager object.
    tmgr = rp.TaskManager(session=session)
    tmgr.add_pilots(pilots)

    # Create a workload of tasks.
    # Each task runs '/bin/date'.
    n = 128  # number of tasks to run
    report.info('create %d task description(s)\n\t' % n)

    tds = list()
    for i in range(0, n):

        # create a new task description, and fill it.
        # Here we don't use dict initialization.
        td = rp.TaskDescription()
        td.executable = '/bin/date'
        tds.append(td)
        report.progress()

    report.ok('>>ok\n')

    # Submit the previously created task descriptions to the
    # PilotManager. This will trigger the selected scheduler to start
    # assigning tasks to the pilots.
    tasks = tmgr.submit_tasks(tds)

    # Wait for all tasks to reach a final state (DONE, CANCELED or FAILED).
    report.header('gather results')
    tmgr.wait_tasks()
    return tasks


We create the <i>report_task_progress</i> function to report the task status of each task

In [ ]:
def report_task_progress(tasks):
    report.info('\n')
    for task in tasks:
        report.plain('  * %s: %s, exit: %3s, out: %s\n'
                % (task.uid, task.state[:4],
                    task.exit_code, task.stdout[:35]))

    # get some more details for one task:
    task_dict = tasks[0].as_dict()
    report.plain("task workdir : %s\n" % task_dict['task_sandbox'])
    report.plain("pilot id     : %s\n" % task_dict['pilot'])
    report.plain("exit code    : %s\n" % task_dict['exit_code'])
    report.plain("stdout       : %s\n" % task_dict['stdout'])

    # get some more details for one task:
    task_dict = tasks[1].as_dict()
    report.plain("task workdir : %s\n" % task_dict['task_sandbox'])
    report.plain("pilot id     : %s\n" % task_dict['pilot'])
    report.plain("exit code    : %s\n" % task_dict['exit_code'])
    report.plain("exit stdout  : %s\n" % task_dict['stdout'])


We put all function calls inside a try except block.  Finally, always clean up the session no matter if we caught an exception or not. This will kill all the remaining pilots.

In [ ]:
try:
    pdesc = initialize_desc_object(resources)
    pilots = launch_pilots(session,pdesc)
    tasks = submit_tasks(pilots)
    report_task_progress(tasks)
except Exception as e:
    # Something unexpected happened in the pilot code above
    report.error('caught Exception: %s\n' % e)
    raise

except (KeyboardInterrupt, SystemExit):
    # the callback called sys.exit(), and we can here catch the
    # corresponding KeyboardInterrupt exception for shutdown.  We also catch
    # SystemExit (which gets raised if the main threads exits for some other
    # reason).
    report.warn('exit requested\n')
finally:
    # always clean up the session, no matter if we caught an exception or
    # not.  This will kill all remaining pilots.
    report.header('finalize')
    session.close()
report.header()